In [14]:
def preprocessor(df1):
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing
    df2=df1[['Net.Due.Dt','Clearing','Status']]
    df2.dropna(inplace=True)
    df2['Range of Delay']=pd.to_datetime(df2['Clearing'],yearfirst=True)-pd.to_datetime(df2['Net.Due.Dt'],yearfirst=True)
    df2['Range of Delay']=df2['Range of Delay'].dt.days
    df2['Range of Delay']=pd.to_numeric(df2['Range of Delay'])
    df2 = df2.reset_index()
    df2.drop(columns='index',inplace=True)
    df2['Status'] = np.where(df2['Range of Delay'] > 180, 8, 
         (np.where(df2['Range of Delay'] > 90, 7,(np.where(df2['Range of Delay'] > 60, 6, (np.where(df2['Range of Delay'] > 30, 5, (np.where(df2['Range of Delay'] > 15, 4, (np.where(df2['Range of Delay'] > 7, 3, (np.where(df2['Range of Delay'] > 3, 2, (np.where(df2['Range of Delay'] > 0, 1 ,0)))))))))))))))
    print(df2.head())
    df1['Status']=df2['Status']
    df1.dropna(inplace=True)
    df1['Status']=df1['Status'].astype('category').cat.codes
    df1['BusA']=df1['BusA'].astype('category').cat.codes
    df1['CCAr']=df1['CCAr'].astype('category').cat.codes
    df1['Account']=df1['Account'].astype('category').cat.codes
    df1['Month']=df1['Month'].astype('category').cat.codes
    #df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1.drop(columns=['DocumentNo','Year','Clrng.doc.'],inplace=True)
    df1['Zone']=df1['Zone'].astype('category').cat.codes
    df1['Bran']=df1['Bran'].astype('category').cat.codes
    df1['PayT'] = df1['PayT'].astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date']).dt.strftime("%Y%m%d").astype(str)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt']).dt.strftime("%Y%m%d").astype(str)
    df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    df1.drop(columns='Clearing',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    y=df1['Status']
    df1.drop(columns='Status',inplace=True)
    print('Y s in the set')
    print(np.unique(y,return_counts=True))
    print('----------------------------------------------------------------------------------------')
    y1=df1.columns
    x = df1.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    return df1,y

In [4]:
def trainer(df1,y):
    import pandas as pd
    import numpy as np
    from sklearn import metrics
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import tree
    from sklearn import neighbors
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import VotingClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.model_selection import train_test_split
    
    #y=np.array(df1['Status'])
    
    

    X=df1
    #df1.head(5)
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state=0)
    # Random forest Model: 
    model= RandomForestClassifier(n_estimators = 300, criterion='entropy',random_state = 0)
    # Decion Tree Model: 
    decision_tree = tree.DecisionTreeClassifier(random_state=0, max_depth=100)
    # AdaBoost Classifier Model:
    clf = AdaBoostClassifier(base_estimator=decision_tree,n_estimators=100, random_state=0)
    # KNN :
    mo = neighbors.KNeighborsClassifier(n_neighbors =2)
    # Bagging Classifier Model:  
    bagging = BaggingClassifier(mo,max_samples=0.5, max_features=0.5)
    #Voting Classifier
    eclf = VotingClassifier(estimators=[ ('knn_bagging', bagging), ('adaboost', clf), ('RF',model)],voting='soft')
    eclf1 = eclf.fit(xtrain, ytrain)
    eclf2=eclf.predict(xtest)
    print('TRAINING RESULTS-----------------------------------------')
    print("Accuracy: for Voting :",metrics.accuracy_score(ytest, eclf2)*100)
    print()
    print('Confusion Matrix for Voting')
    print(metrics.confusion_matrix(eclf2, ytest))
    print()
    print('--------------------------------------------------------------------------------------------------------------------')
    return eclf

In [15]:
def predict(df1,clf):
    import numpy as np
    import pandas as pd
    import copy
    from sklearn import preprocessing
    df=copy.deepcopy(df1)
    print(df.shape)
    df['Prediction']=np.zeros(len(df1))
    df['Pro']=np.zeros(len(df1))
    print(df1.shape)
    df1.dropna(inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    #df1['Status']=df1['Status'].astype('category').cat.codes
    df1['BusA']=df1['BusA'].astype('category').cat.codes
    df1['CCAr']=df1['CCAr'].astype('category').cat.codes
    df1['Account']=df1['Account'].astype('category').cat.codes
    #df1.drop(columns='Reference',inplace=True)
    df1.drop(columns='Customer.Name',inplace=True)
    df1['Month']=df1['Month'].astype('category').cat.codes
    df1.drop(columns=['DocumentNo','Year'],inplace=True)
    df1['Zone']=df1['Zone'].astype('category').cat.codes
    df1['Bran']=df1['Bran'].astype('category').cat.codes
    df1['PayT'] = df1['PayT'].astype('category').cat.codes
    df1.drop(columns='Doc.Chq.dt',inplace=True)
    df1.drop(columns='Ty',inplace=True)
    df1.drop(columns='Sale.Type',inplace=True)
    df1['Pstng.Date']=pd.to_datetime(df1['Pstng.Date']).dt.strftime("%Y%m%d").astype(str)
    df1['Net.Due.Dt']=pd.to_datetime(df1['Net.Due.Dt']).dt.strftime("%Y%m%d").astype(str)
    #df1['Clearing']=pd.to_datetime(df1['Clearing']).dt.strftime("%Y%m%d").astype(str)
    #df1.drop(columns=['Net.Due.Dt','Clearing'],inplace=True)
    #df1.drop(columns='Net.Due.Dt',inplace=True)
    df1 = df1.reset_index()
    df1.drop(columns='index',inplace=True)
    df1.drop(columns='Arr..Clearing...Net.Due.Date.',inplace=True)
    df1.drop(columns='G.L',inplace=True)
    y1=df1.columns
    x = df1.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df1 = pd.DataFrame(x_scaled,columns=df1.columns)
    prediction=clf.predict(df1)
    predictprob=clf.predict_proba(df1)
    #df['Probability']=np.zeros(len(df))
    print(df1.head())
    
    arr=['No delay','0-3 days','4-7 days','8-15 days','16-30 days','31-60 days','61-90 days','90-180 days','>180 days']
    for i in range(len(prediction)):
        df['Prediction'][i]=arr[prediction[i]]
        arr2=""
        for j in range(len(predictprob[i])):
            arr2=arr2+'  Probability of being in ('+str(arr[j])+' delay) is: '+str(predictprob[i][j])+'  , '
        df['Pro'][i]=arr2
            
    print()
    print('-----------------------------------------------------------------------------------------')
    df.to_csv('Predictedout.csv',index=False)

In [16]:
def main():
    import time
    import warnings
    warnings.filterwarnings('ignore')
    import pandas as pd
    import numpy as np
    import copy
    trainpath='MyData.csv'
    testpath='Test.csv'
    print('Reading data...')
    df1=pd.read_csv(trainpath)
    print('Finished Reading...')
    print()
    print(df1.head(5))
    print()
    #df3=copy.deepcopy(df1)
    print('Preprocessing started .......')
    df2,y=preprocessor(df1)
    print(df2.head(5))
    print('Preprocessing Done...........')
    print('--------------------------------------------------------------------------------------------------------')
    t1=time.time()
    print('Training Starting............')
    clf=trainer(df2,y)
    print('Training Done...................')
    print('Time Taken',time.time()-t1)
    print('---------------------------------------------------------------------------------------------------------')
    test=pd.read_csv(testpath)
    predict(test,clf)
    print()
    print('______________________________________ END ____________________________________')
    print()
    print()

In [17]:
main()

Reading data...
Finished Reading...

   BusA CCAr     Reference  Account          Customer.Name   Zone  Bran  Ty  \
0  1000  BPR  4.132000e+09   929488      KAUSALYA AGENCIES  South  SBAD  RV   
1  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
2  9600  BPR  1.115502e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
3  9600  BPR  2.146119e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   
4  2500  BPR  2.410320e+09   928860  TATA STEEL PROCESSING  North  GBAD  RV   

    Status  Local.Crcy.Amt  ...  Doc.Chq.dt Month  Pstng.Date  Net.Due.Dt  \
0   Not OD            0.10  ...  31.12.2018   Dec  2018-12-31  2019-01-05   
1  Overdue            0.04  ...  31.10.2018   Oct  2018-10-31  2018-11-10   
2   Not OD            0.12  ...  05.11.2018   Nov  2018-11-05  2018-11-15   
3  Overdue            0.14  ...  27.10.2018   Oct  2018-10-27  2018-11-06   
4  Overdue            0.16  ...  03.11.2018   Nov  2018-11-03  2018-11-13   

       G.L  Year Clrng.do